# Исследование надежности заемщиков.

### Введение
Заказчиком данного проекта являтеся кредитный отдел банка. Необходимо понять, влияет ли семейное положение, количество детей клиента, уровень дохода и цель кредита на факт погашения кредита в срок.
Полученный результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
Расшифровка столбцов представлена ниже:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("/datasets/data.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Вывод по данным:

В 2х столбцах пропущены данные, а именно:
* `days_emploed` — общий трудовой стаж в днях;
* `total_income` — ежемесячный доход;  
Пропущенные данные можно заменить медианой, а не среднем значением. Так как в столбцах могут присутствовать выбросы и при расчете среднего арифмитического может привести к искожению данных. В то время как медиана возьмет ровно среднее значение. 

### Шаг 2.1 Заполнение пропусков

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

#### Артефакты (аномалии):
В столбце `days_emploed` — общий трудовой стаж в днях наблюдаются отрицательные значения. А также пиковые значения, которые ,скорей всего, в количесвте часов. Отрицательные значения могли быть вызваны неправильной формулой расчета. А пиковые значения занесены/выгружены не в формате "дней". Поэтому для заполнения значений в столбце `days_emploed` необходимо сначала убрать минус и перевести значения из часов в дни.

In [5]:
df["days_employed"] = df["days_employed"].abs()

In [6]:
df[df['days_employed'] > 20000]["children"].count()

3445

In [7]:
# цикл для перевода значений в столбце `days_employed`, который в часах, в дни
for i in range(len(df["days_employed"])):
    try:
        if df["days_employed"][i] > 20000:
            df["days_employed"][i] = df["days_employed"][i] / 24
    except:
        continue

In [8]:
df[df['days_employed'] > 20000]["children"].count()

0

In [9]:
#Код ревьюера
#df[df['days_employed'] > 20000][любой_столбец_без_пропусков].count()

In [10]:
days_employed_median = df["days_employed"].median()
days_employed_median

2194.220566878695

In [11]:
df["days_employed"] = df["days_employed"].fillna(days_employed_median)

In [12]:
total_income_median = df['total_income'].median()
total_income_median

145017.93753253992

In [13]:
df["total_income"] = df["total_income"].fillna(total_income_median)

In [14]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [15]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Шаг 2.2 Проверка данных на аномалии и исправления.

#### Артефакты (аномалии):
* В столбце `days_emploed` — общий трудовой стаж в днях наблюдаются отрицательные значения. А также пиковые значения, которые ,скорей всего, в количесвте часов.
Отрицательные значения могли быть вызваны неправильной формулой расчета. А пиковые значения занесены/выгружены не в формате "дней". 
* В столбце `children` - дети: указано отрицательное значение детей (-1) в 47 случаях. Данная ошибка могла быть вызвана при заполнении в ручную. А также максимальное значение детей достигает 20 в 76 случаях. 
* В столбце `dob_years` присутствуют значения - `0` в 100 строк;

In [16]:
# Убрать артефакты в столбце children.
df["children"].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [17]:
df[df["children"] == 20] = 2

In [18]:
df[df["children"] == -1] = 1

In [19]:
df["children"].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Шаг 2.3. Изменение типов данных.

In [20]:
df["total_income"] = df["total_income"].astype("int")

In [21]:
df["days_employed"] = df["days_employed"].astype("int")

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [23]:
df["education"].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
2                         76
Неоконченное высшее       47
1                         47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [24]:
# Перевод всех значений в нижний регистр
df["education"] = df["education"].str.lower()

In [25]:
df["education"].value_counts()

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

In [26]:
df.duplicated().sum()

192

In [27]:
df = df.drop_duplicates().reset_index(drop = True)

In [28]:
df.duplicated().sum()

0

In [29]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           2
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [30]:
# Значения в education 1 и 2 перешли в пустые строки. Необходимо их удалить
df = df.dropna(subset = ["education"]).reset_index(drop = True)

In [31]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [32]:
# проверка на значений столбца gender
df["gender"].value_counts()

F      14092
M       7238
XNA        1
Name: gender, dtype: int64

In [33]:
# проверка на значений столбца family_status
df["family_status"].value_counts()

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

In [34]:
# проверка на значений столбца income_type
df["income_type"].value_counts()

сотрудник          11015
компаньон           5047
пенсионер           3812
госслужащий         1451
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [35]:
# проверка на значений столбца purpose на неявные дубликаты
df["purpose"].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [36]:
education_dic = df[["education_id",'education']].drop_duplicates().reset_index(drop = True)
family_dic = df[['family_status_id','family_status']].drop_duplicates().reset_index(drop = True)

In [37]:
education_dic

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [38]:
family_dic

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [39]:
# Удаление столбцов в исходном DataFrame
df = df.drop(columns = ['education','family_status'],axis = 1)

### Шаг 2.6. Категоризация дохода.

In [40]:
def total_income_category(row):
    if 0 <= row <= 30000:
        return "E"
    elif 30001 <= row <= 50000:
        return "D"
    elif 50001 <= row <= 200000:
        return "C"
    elif 200001 <= row <= 1000000:
        return "B"
    else:
        return "A"

In [41]:
df["total_income_category"] = df["total_income"].apply(total_income_category)

In [42]:
df["total_income_category"].value_counts()

C    15922
B     5013
D      349
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [43]:
def purpose_category(row):
    if "автомобил" in row:
        return 'операции с автомобилем'
    elif "жиль" in row or "недвиж" in row:
        return "операции с недвижимостью"
    elif "свадьб" in row:
        return 'проведение свадьбы'
    elif "образов" in row:
        return 'получение образования'
    else:
        return row

In [44]:
df['purpose_category'] = df["purpose"].apply(purpose_category)

In [45]:
df["purpose_category"].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:

In [46]:
children_pivot = df.pivot_table(index="children", values ="debt" , aggfunc = ["count","sum","mean"])
children_pivot.columns = ["count","sum","mean"]
children_pivot["procent_vozvrata"] = (1 - children_pivot['mean'])*100
children_pivot

,count,sum,mean,procent_vozvrata
children,,,,
0,14091,1063,0.075438,92.456178
1,4808,444,0.092346,90.765391
2,2052,194,0.094542,90.545809
3,330,27,0.081818,91.818182
4,41,4,0.097561,90.243902
5,9,0,0.000000,100.000000


###### Вывод:
Исходя из сводной таблицы, можно сделать вывод, что:
* Клиенты, у которых количесвто детей равняется `5` - имеют 100% возврат кредита вовремя, однако количесвто таких случаев составляет очень  маленький процент по сравнению с общим. Поэтому данный варинат не подходит для анализа;
* Наиболший процент возврата кредита у клиентов, которых `нет детей` - 92.4%. Скоерй всего, это связано с тем, что у таких клиентов нет дополнительных трат, а все идет на погашение кредита;
* Наихуже всего возвращают кредит клиенты, у которых `4` ребенка в семье, однако количетсво таких случает тоже невилико по сравнению с общим процентом. Поэтому из таблицы можно сделать вывод, что наименьший процент возврата кредита в срок у клиентов, число детей которых равняется `2`. Это связано с тем, что у клиентов идут дополнительные расходы на детей и не всегда удается найти средства для своевременного погашения кредита.

##### Вопрос 2:

In [47]:
new_table_family = df.merge(family_dic, on = "family_status_id", how = 'left' )

In [48]:
new_table_family.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,family_status
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,женат / замужем
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,женат / замужем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,женат / замужем
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,женат / замужем
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,гражданский брак


In [49]:
semya_pivot = new_table_family.pivot_table(index="family_status", values ="debt" , aggfunc = ["count","sum","mean"])
semya_pivot.columns = ["count","sum","mean"]
semya_pivot["procent_vozvrata"] = (1 - semya_pivot['mean'])*100

In [50]:
semya_pivot

,count,sum,mean,procent_vozvrata
family_status,,,,
Не женат / не замужем,2796,273,0.097639,90.236052
в разводе,1189,84,0.070648,92.935240
вдовец / вдова,951,63,0.066246,93.375394
гражданский брак,4134,385,0.093130,90.686986
женат / замужем,12261,927,0.075606,92.439442


###### Вывод:
Исходя из сводной таблицы, можно сделать вывод, что:
* Клиенты c семейным статусом `вдовец / вдова` имеют наибольший процент по возвращению кредита;
* Наихуже всего возвращают кредит клиенты, у которых семейным статус `Не женат / не замужем`;

##### Вопрос 3:

###### Пометка для дохода
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [51]:
income_pivot = df.pivot_table(index="total_income_category", values ="debt" , aggfunc = ["count","sum","mean"])
income_pivot.columns = ["count","sum","mean"]
income_pivot["procent_vozvrata"] = (1 - income_pivot['mean'])*100

In [52]:
income_pivot

,count,sum,mean,procent_vozvrata
total_income_category,,,,
A,25,2,0.080000,92.000000
B,5013,354,0.070616,92.938360
C,15922,1353,0.084977,91.502324
D,349,21,0.060172,93.982808
E,22,2,0.090909,90.909091


###### Вывод:
Исходя из сводной таблицы, можно сделать вывод, что:
* Наибольштй процент возврата кредита у клиентов, чей доход относится к категории `D`;
* Наименьший процент возврата кредита у клиентов, чей доход относится к категории `E`;
* Несмотря на то, что количество в категориях `D`,`E`, `A` мало, по сравнению с категориями `B` и `E`, я считаю что такие данные нельзя исключать из анализа в связи с тем, что люди бывают из разных областей с разным уровнем зароботной платы.

##### Вопрос 4:

In [53]:
purpose_pivot = df.pivot_table(index="purpose_category", values ="debt" , aggfunc =["count","sum","mean"])
purpose_pivot.columns = ["count","sum","mean"]
purpose_pivot["procent_vozvrata"] = (1 - purpose_pivot['mean'])*100

In [54]:
purpose_pivot

,count,sum,mean,procent_vozvrata
purpose_category,,,,
операции с автомобилем,4279,400,0.093480,90.652022
операции с недвижимостью,10751,780,0.072551,92.744861
получение образования,3988,369,0.092528,90.747242
проведение свадьбы,2313,183,0.079118,92.088197


###### Вывод:
Исходя из сводной таблицы, можно сделать вывод, что:
* `операции с недвижимостью` обладают наибольшем процентом по возвращению кредита в срок;
* `операции с автомобилем` занимают последнюю строчку в таблице по возврату кредита;

## Общий вывод:

* На первом этапе проекта были рассмотерны данные и их информация, выявлены пропуски в 2 столбцах и выбросы.
* На втором этапе были убраны выбросы, а пропуски заполнены медианным значением, так как среднне арифмитечское может привести к искожению данных. Далее было обнаружено, что в столбце `education` присутствуют значения в разных регистрах. Такие данные влияют на анализ и поиск дубликатов. Поэтому сначла значения были приведены к одному регистру, а дальше удалены дубликаты. 
* Проверны остальные столбцы на неявные дубликаты. Анализ показал, что неявные дубликаты отсутствуют.
* Для рагрузки общей таблицы и простого восприятия, из главной таблицы убраны столбцы `education` и `family_status` и сделаны отдельные словари для дальнейшего обращения. 
* Для категаризации дохода и целей кредита были написаны функции и применены к основной таблице, для создания новых столбцов.
* Для рассмотрения влияния семейного положения, количество детей клиента, уровень дохода и цели кредита на факт погашения кредита в срок были созданы сводные таблицы.
* Подводя итог, можно сказать, что клиенты у которых нет детей с доходом от 30000 до 50000, бравшие кредит на операции с недвижимостью, имеют наибольший процент возврата кредита в срок. В то время как многодетные клиенты, с доходом до 30000 тыс, бравшие кредит на операции с автомобилем, имеют наименьший процент возврата кредита в срок.